In [125]:
import requests
import time
import csv
import http.client
import json
import pandas as pd
import numpy as np

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

import plotly.express as px
import plotly.graph_objects as go

import country_converter as coco

pd.set_option('display.max_columns', 70)

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "vscode"


# Tutorials used
- Insomnia scraping  https://www.youtube.com/watch?v=DqtlR0y0suo
- geopy https://www.geeksforgeeks.org/how-to-find-longitude-and-latitude-for-a-list-of-regions-or-country-using-python/
- Country converter https://pypi.org/project/country-converter/

# Run next cell to scrape BCL site and put results into .csv

In [2]:
# conn = http.client.HTTPSConnection("www.bcliquorstores.com")

# payload = ""

# ALC_TYPES = ['wine', 'beer', 'spirits', 'coolers/ciders']
# # ALC_TYPES = ['beer']

# results = []


# for ALC in ALC_TYPES:
#     headers = {
#     # 'Cookie': "...",
#     'Accept': "application/json, text/plain, */*",
#     'Accept-Encoding': "gzip, deflate, br",
#     'Host': "www.bcliquorstores.com",
#     # 'User-Agent': "...",
#     'Accept-Language': "en-gb",
#     'Referer': f"https://www.bcliquorstores.com/product-catalogue?category={ALC}&sort=name.raw:asc&page=1",
#     'Connection': "keep-alive"
#     }
#     query_string = f"/ajax/browse?category={ALC}&sort=name.raw%3Aasc&size=1&page=1&="
#     conn.request("GET", query_string, payload, headers)
#     res = conn.getresponse()
#     data = res.read()

#     my_json = data.decode('utf8').replace("'", '"')
#     num_alc = json.loads(my_json)['hits']['total']

#     query_string = f"/ajax/browse?category={ALC}&sort=name.raw%3Aasc&size={num_alc}&page=1&="
#     conn.request("GET", query_string, payload, headers)

#     res = conn.getresponse()
#     data = res.read()
#     my_json = data.decode('utf8').replace("'", '"')
#     json_str = json.loads(my_json)  
#     for p in json_str['hits']['hits']:
#         results.append(p)

# df = pd.json_normalize(results)
# df.to_csv('data/web-scraped-result.csv')

## Run to load info from .csv

In [4]:
df = pd.read_csv('data/web-scraped-result.csv',
                delimiter=',',            # Comma as delimiter
                 quotechar='"',            # Use double quotes for quoting
                quoting=csv.QUOTE_MINIMAL,  # Minimal quoting (or csv.QUOTE_ALL as needed)
                doublequote=True, 
                encoding='utf-8',
                )
df.head()

,Unnamed: 0,_index,_type,_id,_score,_sort,_source.redVarietal,_source.isVQA,_source.subCategory.description,_source.subCategory.id,...,_source.promotions,_source.nameSanitized,_source.isBCVQA,_source._currentPrice,_source._promotionEndDate,_source.votes,_source.category.description,_source.category.id,_source.is_special_release,_source.availability_override
0,0,bcls,product,122867,0.601447,['ABRUZZO PECORINO - BARONE DI VALFORTE 2020'],NaN,NaN,Italy Wine,1043,...,NaN,abruzzo pecorino - barone di valforte 2020,NaN,17.99,2022-12-31T00:00:00.000-08:00,84,Wine,1005,NaN,NaN
1,1,bcls,product,432123,0.601447,['ABRUZZO PINOT GRIGIO - MARE DI SIRENA'],NaN,NaN,Italy Wine,1043,...,"[{'urlAlias': '/promotion/new-arrivals', 'name...",abruzzo pinot grigio - mare di sirena,NaN,12.99,2022-12-31T00:00:00.000-08:00,18,Wine,1005,NaN,NaN
2,2,bcls,product,712331,0.601447,['ACQUESI - ASTI'],NaN,NaN,Italy Wine,1043,...,NaN,acquesi - asti,NaN,18.49,2022-12-31T00:00:00.000-08:00,293,Wine,1005,NaN,NaN
3,3,bcls,product,783498,0.595873,['AGLIANICO DEL VULTURE - BASILISCO TEODOSIO 2...,Aglianico,NaN,Italy Wine,1043,...,NaN,aglianico del vulture - basilisco teodosio 2019,NaN,21.99,2022-12-31T00:00:00.000-08:00,233,Wine,1005,NaN,NaN
4,4,bcls,product,556126,0.601447,['AIX EN PROVENCE ROSE - BIELER PERE & FILS CU...,NaN,NaN,France Wine,1037,...,NaN,aix en provence rose - bieler pere & fils cuve...,NaN,15.97,2022-12-31T00:00:00.000-08:00,116,Wine,1005,NaN,NaN


In [24]:
print(f'Number of columns = {len(df.columns)}')

Number of columns = 56


## Removing categorical columns that are not useful / interesting and have less than 4 values.

In [22]:
boring_cols = []

for col in df.columns:
    unique_vals = df[col].unique()

    if len(unique_vals) < 4:
        print(f'{col} || {unique_vals}')

        boring_cols.append(col)

keep_cols = ['_source.isCraft', 
             '_source.isDealcoholizedWine',
             '_source.isOrganic',
             '_source.isKosher']


boring_cols = list(set(boring_cols) - set(keep_cols))
df.drop(columns = boring_cols, inplace=True)

print(boring_cols)

_index || ['bcls']
_type || ['product']
_source.isVQA || [nan True]
_source._isLimitedTimeOffer || [True nan]
_source.isCraft || [nan True False]
_source.isDraft || [nan False]
_source._promotionStartDate || ['2022-11-27T00:00:00.000-08:00' nan '2022-10-30T00:00:00.000-07:00']
_source.isBCSpirit || [nan]
_source.isLimitedTimeOffer || [True nan]
_source.last_updated || ['2022-11-27 21:10:01']
_source._blacklist || [False]
_source.isDealcoholizedWine || [nan True]
_source.isBCCraft || [False  True]
_source.statusCode || [2 3]
_source.status || [2 3]
_source.isOrganic || [False True nan]
_source.isExclusive || [ True False]
_source.isOntarioVQA || [nan True]
_source.isKosher || [False True nan]
_source._whitelist || [False  True]
_source.inventoryCode || [0 1]
_source.isNew || [False  True]
_source.isBCVQA || [nan True]
_source._promotionEndDate || ['2022-12-31T00:00:00.000-08:00' nan '2022-11-27T00:00:00.000-08:00']
_source.is_special_release || [nan False]
_source.availability_override 

## Count columns with atleast one nan and without any nan's

In [34]:
print(f'Number of columns = {len(df.columns)}')

contains_nan = df.columns[df.isna().any()].tolist()
no_nan = df.columns[df.notna().all()].tolist()

print(f'Number of columns with some nan = {len(contains_nan)}')
print(f'Number of columns with no nan = {len(no_nan)}')

Number of columns = 56
Number of columns with some nan = 22
Number of columns with no nan = 34


### Looking through columns more carefully.

Seeing what has useful information for classifying the product and what are good variables that can be used in analysis. 

In [79]:
cols_to_del = [
    'Unnamed: 0',
    '_id',
    '_source.subCategory.id',
    '_source.promotionStartDate',
    '_source.promotionStartDate',
    '_source.promotionEndDate',
    '_source._all_featured',
    '_source.all_promotions',
    '_source.promotions',
    '_source.sku',
    '_source.sku',
    '_source.image',
    '_source.upc',
    '_source.nameSanitized',
    '_source.category.id',
    '_source.restrictionCode',
    '_source.class.id',
]


df.drop(columns = cols_to_del, inplace=True, errors='ignore')

df.describe(include='all')


,_score,_sort,_source.redVarietal,_source.subCategory.description,_source.consumerRating,_source.featured,_source.regularPrice,_source.isCraft,_source.whiteVarietal,_source.tastingDescription,_source._featured,_source.availableUnits,_source.subRegion,_source.unitSize,_source.isDealcoholizedWine,_source.volume,_source.certificates,_source.name,_source.style,_source.countryName,_source.region,_source.color,_source.grapeType,_source._regularPrice,_source.isOrganic,_source.storeCount,_source.nameSuffix,_source.alcoholPercentage,_source.productCategory,_source.all_featured,_source.productSubCategory,_source.namePrefix,_source.countryCode,_source.class.description,_source.productType,_source.isKosher,_source.currentPrice,_source.sweetness,_source._currentPrice,_source.votes,_source.category.description
count,5548.000000,5548,2051,5548,5548.000000,5548,5548.000000,462,1001,5548,5548,5548.000000,1953,5548.000000,1,5548.000000,5548,5548,460,5548,2758,3523,3523,5548.000000,5546,5529.000000,5547,5548.000000,475,5548,347,5087,5548,5548,5073,5546,5548.000000,2898.000000,5548.000000,5548.000000,5548
unique,NaN,5068,43,39,NaN,8,NaN,2,29,4684,8,NaN,50,NaN,1,NaN,7,5068,26,65,55,4,71,NaN,2,NaN,4500,NaN,3,14,3,1784,65,155,22,2,NaN,NaN,NaN,NaN,5
top,NaN,['FIREBALL'],Bordeaux Blend,France Wine,NaN,[],NaN,False,Other Blend,False,[],NaN,Other,NaN,True,NaN,[],FIREBALL,Lager,France,Bordeaux,RED,BORDEAUX BLEND,NaN,False,NaN,12 YEAR OLD,NaN,Domestic Beer,[],BC Craft Beer,CABERNET SAUVIGNON,FR,France Red Wine,Red Wine,False,NaN,NaN,NaN,NaN,Wine
freq,NaN,7,563,1277,NaN,5367,NaN,301,266,278,5367,NaN,607,NaN,1,NaN,5093,7,157,1387,610,2051,611,NaN,5450,NaN,26,NaN,347,3172,206,187,1387,755,2021,5508,NaN,NaN,NaN,NaN,3530
mean,0.498403,NaN,NaN,NaN,3.593980,NaN,216.672383,NaN,NaN,NaN,NaN,1454.248198,NaN,1.777758,NaN,0.843881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.672383,NaN,69.521432,NaN,18.977648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215.900339,0.700138,215.900339,75.382300,NaN
std,0.136803,NaN,NaN,NaN,1.271495,NaN,1549.303348,NaN,NaN,NaN,NaN,2966.984517,NaN,3.009659,NaN,1.017603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549.303348,NaN,65.518562,NaN,13.192768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549.375563,2.081728,1549.375563,103.675391,NaN
min,0.200588,NaN,NaN,NaN,0.000000,NaN,1.990000,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000,NaN,0.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.990000,NaN,1.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.990000,0.000000,1.990000,0.000000,NaN
25%,0.362997,NaN,NaN,NaN,3.400000,NaN,19.990000,NaN,NaN,NaN,NaN,67.000000,NaN,1.000000,NaN,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.990000,NaN,7.000000,NaN,12.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.990000,0.000000,18.990000,7.000000,NaN
50%,0.595873,NaN,NaN,NaN,3.900000,NaN,35.490000,NaN,NaN,NaN,NaN,574.000000,NaN,1.000000,NaN,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.490000,NaN,49.000000,NaN,13.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.990000,0.000000,34.990000,41.000000,NaN
75%,0.601447,NaN,NaN,NaN,4.300000,NaN,87.990000,NaN,NaN,NaN,NaN,1768.250000,NaN,1.000000,NaN,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.990000,NaN,123.000000,NaN,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.990000,0.000000,86.990000,107.000000,NaN


In [91]:

df['_source.productType'].unique() # all other than beer
df['_source.productCategory'].unique() # beer
df['_source.style'].unique() # beer style e.g. fuit ale, lager, stout

# merge white and red wine variety
df['_source.whiteVarietal'].unique() 
df['_source.redVarietal'].unique() 


df['_source.productType'].unique() # general description (not bad)

## RATING               _source.consumerRating
## SCORE                _score
## VOTES                _source.votes
## PRICE                _source.regularPrice
## REVIEW               _source.tastingDescription
## AVAILABLE_UNITS      _source.availableUnits   
## CERTIFICATES         _source.certificates
## NAME                 _source.name
## BEER STYLE           _source.style'
## GENERAL DESCRIPTION  _source.subCategory.description'
## ALCOHOL PERCENTAGE   _source.alcoholPercentage
## SWEETNESS            _source.sweetness
## COUNTRY              _source.countryName

array(['White Wine', 'Sparkling Wine', 'Red Wine', 'Rose Wine', 'Sherry',
       'Other Wine', 'Sake', 'Port', 'Ice Wine', 'Vermouth',
       'De-Alcoholized Wine', nan, 'Other Spirits', 'Whisky / Whiskey',
       'Tequila', 'Liqueurs', 'Cognac', 'Vodka', 'Rum', 'Gin', 'Brandy',
       'Coolers', 'Cider'], dtype=object)

## Defining the columns to keep

In [92]:
COLS_TO_KEEP = [
            '_source.consumerRating',
            '_score',
            '_source.votes',
            '_source.regularPrice',
            '_source.tastingDescription',
            '_source.availableUnits',
            '_source.certificates',
            '_source.name',
            '_source.style',
            '_source.subCategory.description',
            '_source.alcoholPercentage',
            '_source.sweetness',
            '_source.countryName'
]


new_df = df[COLS_TO_KEEP]
new_df.head(3)


,_source.consumerRating,_score,_source.votes,_source.regularPrice,_source.tastingDescription,_source.availableUnits,_source.certificates,_source.name,_source.style,_source.subCategory.description,_source.alcoholPercentage,_source.sweetness,_source.countryName
0,3.5,0.601447,84,19.99,"90 Points, Gismondi On Wine: ""This pecorino is...",887,[],ABRUZZO PECORINO - BARONE DI VALFORTE 2020,NaN,Italy Wine,13.0,0.0,Italy
1,4.1,0.601447,18,14.99,This classic Pinot Grigio shows flavours of ci...,2723,[],ABRUZZO PINOT GRIGIO - MARE DI SIRENA,NaN,Italy Wine,12.5,0.0,Italy
2,3.6,0.601447,293,20.99,An aromatic and sweet white sparkling wine. Pa...,2306,[],ACQUESI - ASTI,NaN,Italy Wine,7.0,8.0,Italy


## Assigning a latitude and longitude to the unique countries

In [94]:
longitude = []
latitude = []

countries = new_df['_source.countryName'].unique()
num_countries = len(countries)

def findGeocode(city):
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="bcl-webscrape")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)  

count = 1
for i in (countries):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)
    
    print(f'Logging {i}, {count}/{num_countries}')
    count += 1

Logging Italy, 1/65
Logging France, 2/65
Logging Spain, 3/65
Logging Portugal, 4/65
Logging Greece, 5/65
Logging Canada, 6/65
Logging Argentina, 7/65
Logging USA, 8/65
Logging Japan, 9/65
Logging Australia, 10/65
Logging Luxembourg, 11/65
Logging Germany, 12/65
Logging South Africa, 13/65
Logging Korea - South, 14/65
Logging Austria, 15/65
Logging Uruguay, 16/65
Logging Chile, 17/65
Logging China, 18/65
Logging Israel, 19/65
Logging New Zealand, 20/65
Logging Lebanon, 21/65
Logging Croatia, 22/65
Logging Turkey, 23/65
Logging United Kingdom, 24/65
Logging Hungary, 25/65
Logging Georgia, 26/65
Logging Bulgaria, 27/65
Logging Switzerland, 28/65
Logging Montenegro, 29/65
Logging Morocco, 30/65
Logging Czech Republic, 31/65
Logging Netherlands, 32/65
Logging Iceland, 33/65
Logging Trinidad And Tobago, 34/65
Logging Ireland, 35/65
Logging Denmark, 36/65
Logging Mexico, 37/65
Logging Belgium, 38/65
Logging Philippines, 39/65
Logging Thailand, 40/65
Logging Singapore, 41/65
Logging Poland, 42

## Assigning a country code

In [95]:
standard_names02 = coco.convert(names=countries, to='ISO2') 
standard_names03 = coco.convert(names=countries, to='ISO3') 

country_df = pd.DataFrame({
    'country':countries,
    'lat':latitude,
    'lon':longitude,
    'country_code02':standard_names02,
    'country_code03':standard_names03
})
print(country_df)

Virgin Islands not found in regex
Virgin Islands not found in regex


      country        lat         lon country_code02 country_code03
0       Italy  42.638426   12.674297             IT            ITA
1      France  46.603354    1.888334             FR            FRA
2       Spain  39.326068   -4.837979             ES            ESP
3    Portugal  39.662165   -8.135352             PT            PRT
4      Greece  38.995368   21.987713             GR            GRC
..        ...        ...         ...            ...            ...
60    Vietnam  15.926666  107.965086             VN            VNM
61   Ukraine   49.487197   31.271832             UA            UKR
62       Fiji -18.123970  179.012274             FJ            FJI
63  Guatemala  15.585555  -90.345759             GT            GTM
64     Latvia  56.840649   24.753764             LV            LVA

[65 rows x 5 columns]


## Adding latitiude, longitude and country code to large dataframe

In [116]:
# lat_mat = pd.DataFrame(new_df.apply (lambda row: country_df['lat'].loc[country_df['country'] == row['_source.countryName']], axis=1))
# lon_mat = pd.DataFrame(new_df.apply (lambda row: country_df['lon'].loc[country_df['country'] == row['_source.countryName']], axis=1))
# coco_mat02 = pd.DataFrame(new_df.apply (lambda row: country_df['country_code02'].loc[country_df['country'] == row['_source.countryName']], axis=1))
# coco_mat03 = pd.DataFrame(new_df.apply (lambda row: country_df['country_code03'].loc[country_df['country'] == row['_source.countryName']], axis=1))

lat_mat = pd.DataFrame(new_df.apply(lambda row: country_df.loc[country_df['country'] == row['_source.countryName'], 'lat'].values[0], axis=1))
lon_mat = pd.DataFrame(new_df.apply(lambda row: country_df.loc[country_df['country'] == row['_source.countryName'], 'lon'].values[0], axis=1))
coco_mat02 = pd.DataFrame(new_df.apply(lambda row: country_df.loc[country_df['country'] == row['_source.countryName'], 'country_code02'].values[0], axis=1))
coco_mat03 = pd.DataFrame(new_df.apply(lambda row: country_df.loc[country_df['country'] == row['_source.countryName'], 'country_code03'].values[0], axis=1))

temp_lat = lat_mat
temp_lon = lon_mat
temp_coco02 = coco_mat02
temp_coco03 = coco_mat03
for index in range(int(np.shape(lat_mat)[1])):
    temp_lat = pd.DataFrame(temp_lat[0].fillna(lat_mat[index]))
    temp_lon = pd.DataFrame(temp_lon[0].fillna(lon_mat[index]))
    temp_coco02 = pd.DataFrame(temp_coco02[0].fillna(coco_mat02[index]))
    temp_coco03 = pd.DataFrame(temp_coco03[0].fillna(coco_mat03[index]))

# new_df['lat'] = temp_lat
# new_df['lon'] = temp_lon
# new_df['country_code02'] = temp_coco02
# new_df['country_code03'] = temp_coco03
new_df.loc[:, 'lat'] = temp_lat.squeeze()
new_df.loc[:, 'lon'] = temp_lon.squeeze()
new_df.loc[:, 'country_code02'] = temp_coco02.squeeze()
new_df.loc[:, 'country_code03'] = temp_coco03.squeeze()

In [138]:
new_df.head(3)

print(f'Number of columns = {len(new_df.columns)}')
print(f'Number of rows = {new_df.shape[0]}')

new_df.describe(include='object')



Number of columns = 17
Number of rows = 5548


,_source.tastingDescription,_source.certificates,_source.name,_source.style,_source.subCategory.description,_source.countryName,country_code02,country_code03
count,5548,5548,5548,460,5548,5548,5548,5548
unique,4684,7,5068,26,39,65,65,65
top,False,[],FIREBALL,Lager,France Wine,France,FR,FRA
freq,278,5093,7,157,1277,1387,1387,1387


In [142]:
len(new_df['_source.tastingDescription'].unique())

4684

## Correlation matrix

In [128]:
COLS_FOR_CORR = ['_source.consumerRating',
            '_score',
            '_source.votes',
            '_source.regularPrice',
            '_source.availableUnits',
            '_source.alcoholPercentage',
            '_source.sweetness']


# Calculate correlation matrix
corr_matrix = new_df[COLS_FOR_CORR].corr()

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,  # Correlation values
    x=corr_matrix.columns,  # Column names for x-axis
    y=corr_matrix.columns,  # Column names for y-axis
    colorscale='RdBu',   # Color scale
    zmin=-1, zmax=1,        # Set the color range from -1 to 1
))

# Add a title to the heatmap
fig.update_layout(title='Correlation Matrix', 
                  xaxis_nticks=36,
                  width=800,
                 height=600, )


# Show the heatmap
fig.show()



## Saving dataframe into .csv so we can do analysis on it later

In [144]:
new_df.to_csv('data/for-analysis.csv', index=False) 


## Counting alcohol per country

In [60]:
num_p_country = np.zeros(len(country_df['country'].index))

for index, country in enumerate(country_df['country']):
    num = len(df.loc[df['_source.countryName'] == country].index)
    num_p_country[index] = num

country_df['num_p_country'] = num_p_country

## Plotting interactive graph to visualise where BCL products come from

https://plotly.com/python/map-configuration/

In [61]:
fig = go.Figure(data=go.Choropleth(
    locations = country_df['country_code03'],
    z = country_df['num_p_country'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'No. Products',
))

fig.update_layout(
    title_text='BCL products from country',
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)


fig.show(renderer="notebook_connected")

## Useful Links for plotting these graphs

https://towardsdatascience.com/work-with-geospatial-data-and-create-interactive-maps-using-geopy-and-plotly-28178d2868f1 </br>
https://plotly.com/python/choropleth-maps/ </br>
https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv </br>
https://www.infragistics.com/help/reveal/location-data-requirements </br>